In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense
from keras import optimizers

In [2]:
storage_account_name = "<Storage account name>"
storage_account_key = "<Storage account key>"
container = "<Container name>"

In [3]:
spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name), storage_account_key)

In [4]:
data = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ",") \
  .csv("wasbs://{0}@{1}.blob.core.windows.net/housing.csv".format(container, storage_account_name))

data.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|
 -122.25| 37.85| 52.0| 919.0| 213.0| 413.0| 193.0| 4.0368| 269700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 2535.0| 489.0| 1094.0| 514.0| 3.6591| 299200.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3104.0| 687.0| 1157.0| 647.0| 3.12| 241400.0| NEAR BAY|
 -122.26| 37.84| 42.0| 2555.0| 665.0| 1206.0| 595.0| 2.0804| 226700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3549.0| 707.0| 1551.0| 714.0| 3.6912| 261100.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2202.0| 434.0| 910.0| 402.0| 3.2031| 281500.0| NEAR BAY|
 -122.26| 37.85| 52.0| 3503.0| 752.0| 1504.0| 734.0| 3.2705| 241800.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2491.0| 474.0| 1098.0| 468.0| 3.075| 213500.0| NEAR BAY|
 -122.26| 37.84| 52.0| 696.0| 191.0| 345.0| 174.0| 2.6736| 191300.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2643.0| 626.0| 1212.0| 620.0| 1.9167| 159200.0| NEAR BAY|
 -122.26| 37.85| 50.0| 1120.0| 283.0| 697.0| 264.0| 2.125| 140000.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1966.0| 347.0| 793.0| 331.0| 2.775| 152500.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1228.0| 293.0| 648.0| 303.0| 2.1202| 155500.0| NEAR BAY|
 -122.26| 37.84| 50.0| 2239.0| 455.0| 990.0| 419.0| 1.9911| 158700.0| NEAR BAY|
 -122.27| 37.84| 52.0| 1503.0| 298.0| 690.0| 275.0| 2.6033| 162900.0| NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 20 rows

In [5]:
df = data.toPandas()
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
df = pd.get_dummies(df, columns=["ocean_proximity"])

In [7]:
df = df.dropna(axis="rows")

In [8]:
dim = df.drop("median_house_value", axis="columns").shape[1]

model = Sequential()
model.add(Dense(8, input_dim=dim, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(2, activation="relu"))
model.add(Dense(1, activation="linear"))

In [9]:
model.compile(loss="mean_absolute_percentage_error", optimizer="rmsprop")

In [10]:
model.fit(df.drop("median_house_value", axis="columns"), df["median_house_value"])

Epoch 1/1

 32/20433 [..............................] - ETA: 7:57 - loss: 100.0000
 704/20433 [>.............................] - ETA: 22s - loss: 100.0000 
 1632/20433 [=>............................] - ETA: 9s - loss: 100.0000 
 2560/20433 [==>...........................] - ETA: 6s - loss: 99.9999 
 3392/20433 [===>..........................] - ETA: 4s - loss: 99.9999
 4352/20433 [=====>........................] - ETA: 3s - loss: 99.9998
 5312/20433 [======>.......................] - ETA: 2s - loss: 99.9828
 6016/20433 [=======>......................] - ETA: 2s - loss: 99.9606
 6880/20433 [=========>....................] - ETA: 2s - loss: 99.9267
 7776/20433 [==========>...................] - ETA: 1s - loss: 99.8851
 8640/20433 [===========>..................] - ETA: 1s - loss: 99.8397
 9568/20433 [=============>................] - ETA: 1s - loss: 99.7787
10528/20433 [==============>...............] - ETA: 1s - loss: 99.7050
11488/20433 [===============>..............] - ETA: 1s - loss: 99.6009
12352/20433 [=================>............] - ETA: 0s - loss: 99.5042
12896/20433 [=================>............] - ETA: 0s - loss: 99.4353
13856/20433 [===================>..........] - ETA: 0s - loss: 99.3034
14816/20433 [====================>.........] - ETA: 0s - loss: 99.1491
15744/20433 [======================>.......] - ETA: 0s - loss: 98.9772
16640/20433 [=======================>......] - ETA: 0s - loss: 98.7990
16896/20433 [=======================>......] - ETA: 0s - loss: 98.7374
17696/20433 [========================>.....] - ETA: 0s - loss: 98.5491
18624/20433 [==========================>...] - ETA: 0s - loss: 98.3028
19392/20433 [===========================>..] - ETA: 0s - loss: 98.0759
20320/20433 [============================>.] - ETA: 0s - loss: 97.7801
20433/20433 [==============================] - 2s 98us/step - loss: 97.7413
Out[45]: <keras.callbacks.History at 0x7f65dc433208>